# Kahve Tüketimi ve Stres Analizi (Saf Versiyon - Bias-Free)

Bu analizde, "Kahve içme nedeni?" sütunu analizden çıkarılarak, stres ve kahve tüketimi arasındaki ilişki daha objektif bir şekilde incelenmiştir. Bu sayede "Stresle başa çıkma" cevabının yaratabileceği döngüsel mantık (Hali hazırda stresli olduğu için kahve içme durumu) elenmiştir.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Veri Yükleme ve Temizleme
print("="*80)
print("KAHVE TÜKETİMİ VE STRES İLİŞKİSİ - SAF ANALİZ")
print("="*80)

df = pd.read_csv('anket.csv')
print(f"\n✓ Veri yüklendi: {df.shape[0]} katılımcı, {df.shape[1]} sütun")

# Gereksiz ve yanlılık yaratabilecek sütunların çıkarılması
drop_columns = ['Zaman damgası', 'Kahve içme nedeni? ']
df_clean = df.drop(drop_columns, axis=1)

print(f"\n⚠️ 'Kahve içme nedeni?' sütunu analizden çıkarıldı (Bias Önleme).")

# Sütun isimlerini düzenleme
df_clean.columns = ['Yas', 'Cinsiyet', 'Is_Yogunlugu', 'Kahve_Miktar', 
                     'Kahve_Zamani', 'Kahve_Hissi', 'Stres_Duzeyi', 
                     'Uyku_Suresi', 'Ruh_Hali', 'Stresli_Kahve']

def simplify(value):
    if pd.isna(value):
        return value
    if ',' in str(value):
        return str(value).split(',')[0].strip()
    return str(value)

df_ml = df_clean.copy()
for col in ['Kahve_Zamani', 'Kahve_Hissi', 'Uyku_Suresi', 'Ruh_Hali']:
    df_ml[col] = df_ml[col].apply(simplify)

# Label Encoding
df_encoded = df_ml.copy()
for column in df_encoded.columns:
    if df_encoded[column].dtype == 'object':
        le = LabelEncoder()
        df_encoded[column] = le.fit_transform(df_encoded[column].astype(str))

In [ ]:
# ADIM 1: KORELASYON ANALİZİ
print("\n" + "="*80)
print("ADIM 1: KORELASYON ANALİZİ")
print("="*80)

correlation_matrix = df_encoded.corr()
stres_correlations = correlation_matrix['Stres_Duzeyi'].sort_values(ascending=False)

print("\n📊 Stres Düzeyi ile Diğer Değişkenler Arasındaki Korelasyon:")
print("-"*80)
for feature, corr_value in stres_correlations.items():
    if feature != 'Stres_Duzeyi':
        if abs(corr_value) >= 0.3:
            guc = "🔴 GÜÇLÜ"
        elif abs(corr_value) >= 0.15:
            guc = "🟡 ORTA "
        else:
            guc = "🟢 ZAYIF"
        print(f"{guc} - {feature:20s}: {corr_value:+.4f}")

In [ ]:
# ADIM 2: OZELLIK ONEM ANALİZİ (Feature Importance)
print("\n" + "="*80)
print("ADIM 2: FEATURE IMPORTANCE ANALİZİ (Random Forest)")
print("="*80)

X = df_encoded.drop('Stres_Duzeyi', axis=1)
y = df_encoded['Stres_Duzeyi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

rf_importance = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=10)
rf_importance.fit(X_train, y_train)

feature_imp_df = pd.DataFrame({
    'Ozellik': X.columns,
    'Onem_Skoru': rf_importance.feature_importances_
}).sort_values('Onem_Skoru', ascending=False)

print("\n📊 Özellik Önem Skorları:")
print("-"*80)
for idx, row in feature_imp_df.iterrows():
    bar = '█' * int(row['Onem_Skoru'] * 100)
    if row['Onem_Skoru'] > 0.10:
        onem = "🔴 ÇOK ÖNEMLİ"
    elif row['Onem_Skoru'] > 0.05:
        onem = "🟡 ÖNEMLİ    "
    else:
        onem = "🟢 AZ ÖNEMLİ "
    print(f"{onem} - {row['Ozellik']:20s}: {row['Onem_Skoru']:.4f} {bar}")

In [ ]:
# ADIM 3: KARŞILAŞTIRMA VE SEÇİM
print("\n" + "="*80)
print("ADIM 3: KORELASYON + FEATURE IMPORTANCE KARŞILAŞTIRMASI")
print("="*80)

comparison_df = pd.DataFrame({
    'Ozellik': X.columns,
    'Korelasyon': [stres_correlations[col] for col in X.columns],
    'Feature_Importance': rf_importance.feature_importances_
})
comparison_df['Abs_Korelasyon'] = comparison_df['Korelasyon'].abs()
comparison_df = comparison_df.sort_values('Feature_Importance', ascending=False)

print("\n📊 Her İki Analize Göre Özellik Değerlendirmesi:")
print("-"*80)
print(f"{'Özellik':<20} {'Korelasyon':>12} {'Feature Imp':>15} {'Karar':>15}")
print("-"*80)

onemli_ozellikler = []
onemsiz_ozellikler = []

for _, row in comparison_df.iterrows():
    ozellik = row['Ozellik']
    korr = row['Abs_Korelasyon']
    fi = row['Feature_Importance']
    
    if korr >= 0.10 or fi >= 0.05:
        karar = "✅ KULLAN"
        onemli_ozellikler.append(ozellik)
    else:
        karar = "❌ ÇIKAR"
        onemsiz_ozellikler.append(ozellik)
    
    print(f"{ozellik:<20} {row['Korelasyon']:>+12.4f} {fi:>15.4f} {karar:>15}")

print(f"\n✅ Kullanılacak özellik sayısı: {len(onemli_ozellikler)}")
if len(onemsiz_ozellikler) > 0:
    print(f"❌ Çıkarılacak özellikler: {', '.join(onemsiz_ozellikler)}")
    print(f"   (Düşük korelasyon ve feature importance nedeniyle)")

In [ ]:
# ADIM 4: MODEL EĞİTİMİ (Seçilen Özelliklerle)
print("\n" + "="*80)
print("ADIM 4: SEÇİLEN ÖZELLİKLERLE MODEL EĞİTİMİ")
print("="*80)

X_selected = X[onemli_ozellikler]
X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nOrijinal özellik sayısı: {X.shape[1]}")
print(f"Seçilen özellik sayısı: {len(onemli_ozellikler)}")
print(f"Azaltma oranı: %{(1 - len(onemli_ozellikler)/X.shape[1])*100:.1f}")

# Model 1: KNN
print("\n1. K-NEAREST NEIGHBORS (KNN)")
print("-"*40)
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train_sel, y_train_sel)
knn_acc = accuracy_score(y_test_sel, knn.predict(X_test_sel))
print(f"   Doğruluk: %{knn_acc*100:.2f}")

# Model 2: Decision Tree
print("\n2. DECISION TREE")
print("-"*40)
dt = DecisionTreeClassifier(max_depth=8, random_state=42)
dt.fit(X_train_sel, y_train_sel)
dt_acc = accuracy_score(y_test_sel, dt.predict(X_test_sel))
print(f"   Doğruluk: %{dt_acc*100:.2f}")

# Model 3: Random Forest
print("\n3. RANDOM FOREST")
print("-"*40)
rf = RandomForestClassifier(n_estimators=150, max_depth=6, random_state=42)
rf.fit(X_train_sel, y_train_sel)
rf_acc = accuracy_score(y_test_sel, rf.predict(X_test_sel))
print(f"   Doğruluk: %{rf_acc*100:.2f}")

# SONUÇLAR
models = [('KNN', knn_acc), ('Decision Tree', dt_acc), ('Random Forest', rf_acc)]
best_model = max(models, key=lambda x: x[1])

print("\n" + "="*80)
print("SONUÇLAR")
print("="*80)

print("\n📊 MODEL PERFORMANSLARI:")
for model_name, acc in sorted(models, key=lambda x: x[1], reverse=True):
    star = "🏆 " if model_name == best_model[0] else "   "
    print(f"{star}{model_name:20s}: %{acc*100:.2f}")

print(f"\n🏆 EN İYİ MODEL: {best_model[0]} (%{best_model[1]*100:.2f})")